In [1]:
import snowflake.connector
import pandas
import torch
from interactions import Interactions
from cross_validation import random_train_test_split

In [2]:
import snowflake.connector
import pandas as pd

def create_connection():
    ctx = snowflake.connector.connect(
        user='Zan',
        password='Kundalini55!',
        account='livenpay.ap-southeast-2')
    return ctx

def setup_snowflake(cursor):
    cursor.execute("use warehouse zans_wh")
    cursor.execute("use zans_db.postgres")
    
def query(cursor,query):
    cursor.execute(query)
    results = cursor.fetchall()
    return results

timestamped_sql = """
SELECT 
	accountcoupon_couponusehistory.time AS "accountcoupon_couponusehistory.time",
	account_localuser.id  AS "account_localuser.id",
	account_localuser.email  AS "account_localuser.email",
	merchant_merchant.name  AS "merchant_merchant.name",
	case when merchant_merchant.membership_zone_id = 1 then 'Melbourne'
    when merchant_merchant.membership_zone_id = 2 then 'Sydney'
    else null end AS "merchant_merchant.city",
	branch_branch.id  AS "branch_branch.id",
	branch_branch.suburb_id  AS "branch_branch.suburb_id"
FROM account_localuser AS account_localuser
LEFT JOIN accountcoupon_couponusehistory  AS accountcoupon_couponusehistory ON account_localuser.id = accountcoupon_couponusehistory.user_id 
LEFT JOIN branch_branch  AS branch_branch ON accountcoupon_couponusehistory.branch_id = branch_branch.id 
LEFT JOIN merchant_merchant  AS merchant_merchant ON merchant_merchant.id = branch_branch.merchant_id 

WHERE ((CASE WHEN account_localuser.is_staff  THEN 1 ELSE 0 END
) = 0) AND ((CASE
    WHEN DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())) is not null
    AND DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))) is not null /* date ranges or in the past x days */
    THEN
    CASE
    WHEN accountcoupon_couponusehistory.time >=  TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'This Period'
    WHEN accountcoupon_couponusehistory.time >=
    DATEADD('day',
      DATEDIFF('day',
        TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))),
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )*-1,
      DATEADD('day', -1,
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )
    )
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('day', 1,DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'Previous Period'
    END
    END
) IS NOT NULL
)
GROUP BY 7,1,2,3,4,5,6
ORDER BY 1 DESC
"""

sql = """
SELECT 
	account_localuser.id  AS "account_localuser.id",
	account_localuser.email  AS "account_localuser.email",
	merchant_merchant.name  AS "merchant_merchant.name",
	case when merchant_merchant.membership_zone_id = 1 then 'Melbourne'
    when merchant_merchant.membership_zone_id = 2 then 'Sydney'
    else null end AS "merchant_merchant.city",
	branch_branch.id  AS "branch_branch.id",
	branch_branch.suburb_id  AS "branch_branch.suburb_id"
FROM account_localuser AS account_localuser
LEFT JOIN accountcoupon_couponusehistory  AS accountcoupon_couponusehistory ON account_localuser.id = accountcoupon_couponusehistory.user_id 
LEFT JOIN branch_branch  AS branch_branch ON accountcoupon_couponusehistory.branch_id = branch_branch.id 
LEFT JOIN merchant_merchant  AS merchant_merchant ON merchant_merchant.id = branch_branch.merchant_id 

WHERE ((CASE WHEN account_localuser.is_staff  THEN 1 ELSE 0 END
) = 0) AND ((CASE
    WHEN DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())) is not null
    AND DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))) is not null /* date ranges or in the past x days */
    THEN
    CASE
    WHEN accountcoupon_couponusehistory.time >=  TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'This Period'
    WHEN accountcoupon_couponusehistory.time >=
    DATEADD('day',
      DATEDIFF('day',
        TO_TIMESTAMP(DATEADD('month', 12, DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))),
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )*-1,
      DATEADD('day', -1,
        TO_TIMESTAMP(DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE())))
      )
    )
    AND accountcoupon_couponusehistory.time <= TO_TIMESTAMP(DATEADD('day', 1,DATEADD('month', -11, DATE_TRUNC('month', CURRENT_DATE()))))
    THEN 'Previous Period'
    END
    END
) IS NOT NULL
)
GROUP BY 1,2,3,4,5,6
ORDER BY 1 DESC
"""

ctx = create_connection()
cursor = ctx.cursor()
setup_snowflake(cursor)

TRANSACTIONS = query(cursor,sql)

cursor.close()
ctx.close()

all_data = pd.DataFrame(TRANSACTIONS, columns=['userId','email','merchant','city','branchId','suburb'])
#all_data = pd.DataFrame(TRANSACTIONS, columns=['time','userId','email','merchant','city','branchId','suburb'])

In [3]:
CITY = "Sydney"

cull = ['Uber','Secure Parking Melbourne','Taste Of Melbourne','UberEATS Sydney','Liven Bistro'
       , 'Cardly - You Write, We Post', 'Good Food & Wine Show', 'Sayers Sister [Kounta Demo]'
       , 'Airtasker','Peninsula Hot Springs','Melbourne River Cruises','Grilla House','Margaret River Gourmet Escape','Madeira','Royale Fusion'
        ,'The Bar.Ber','Tandoori Point','The Italian Cucina','Tokyo Bar','The Trust'
        ,"Orita's",'Pelican ','Olive Garden','Golftec','Lash Labs','Cafe on Bourke','Ibuki House','Mint Leaf',"Tina's Noodle Kitchen"
       ,'Uber Sydney','Cardly - You Write, We Post','Secure Parking Sydney','UberEATS Sydney','Liven Bistro','Good Food & Wine Show','Sayers Sister [Kounta Demo]','Airtasker','Taste Of Sydney']

for merchant in cull:
    all_data = all_data[all_data.merchant!=merchant]
    
all_data = all_data.dropna()
all_data.branchId = all_data.branchId.astype(int)
city_data = all_data[all_data.city==CITY]
city_data.head()

,userId,email,merchant,city,branchId,suburb
3,366067,bdm@rnjrealty.com.au,Ajisen Ramen,Sydney,1482,411.0
4,366064,preciousngo@hotmail.com,CoCo Fresh Tea & Juice,Sydney,1246,418.0
13,366045,elizamcgowan@icloud.com,Messina,Sydney,1113,411.0
14,366044,ryan@lonstein.net,BL Burgers,Sydney,1494,422.0
15,366041,jess.lee.phillips@live.com.au,Oh! Matcha,Sydney,1459,411.0


In [4]:
userId,branchId,suburb = city_data['userId'],city_data['branchId'],city_data['suburb']
interactions = pd.DataFrame({"userId":userId,"branchId":branchId,"suburb":suburb})
interactions = interactions[interactions.groupby('userId').userId.transform(len) > 1]

interactions.head()

,userId,branchId,suburb
66,365916,1458,411.0
67,365916,1247,411.0
68,365914,1575,411.0
69,365914,1115,489.0
104,365427,1538,421.0


In [5]:
from collections import defaultdict

userIds = interactions.userId.unique()
itemIds = interactions.branchId.unique()
internal_userIds = defaultdict(lambda: len(internal_userIds))
internal_itemIds = defaultdict(lambda: len(internal_itemIds))

external_userIds = {}
external_itemIds = {}

for user,item in zip(interactions.userId.values,interactions.branchId.values):
    internal_userIds[user]
    internal_itemIds[item]
    external_userIds[internal_userIds[user]] = user
    external_itemIds[internal_itemIds[item]] = item

In [6]:
interactions.head()

,userId,branchId,suburb
66,365916,1458,411.0
67,365916,1247,411.0
68,365914,1575,411.0
69,365914,1115,489.0
104,365427,1538,421.0


In [7]:
interactions_proc = interactions

internalItems = []
internalUsers = []

for i,r in interactions.iterrows():
    internalItems += [internal_itemIds[int(r.branchId)]]
    internalUsers += [internal_userIds[int(r.userId)]]
    
interactions_proc['branchId'] = internalItems
interactions_proc['userId'] = internalUsers

In [8]:
interactions_proc.loc[interactions_proc['userId']==1]

,userId,branchId,suburb
68,1,2,411.0
69,1,3,489.0


In [9]:
userIds = interactions_proc.userId.values
itemIds = interactions_proc.branchId.values
suburbs = interactions_proc.suburb.values

In [10]:
interactions_proc.head()

,userId,branchId,suburb
66,0,0,411.0
67,0,1,411.0
68,1,2,411.0
69,1,3,489.0
104,2,4,421.0


In [11]:
featureDict = {}

for i in itemIds:
    featureDict[i] = int(interactions_proc.loc[interactions_proc['branchId']==i]['suburb'].unique())

In [12]:
item_features = list(featureDict.values())

In [13]:
type(userIds)

numpy.ndarray

In [14]:
import numpy as np
feature_interactions = Interactions(userIds,itemIds,item_features=np.asarray(item_features))

In [15]:
feature_interactions

<Interactions dataset (13508 users x 314 items x 51002 interactions)>

In [16]:
from scipy import sparse

user_identity = sparse.identity(no_features.num_users)
item_identity = sparse.identity(no_features.num_items)

NameError: name 'no_features' is not defined

In [17]:
feature_interactions.item_features

array([  411.,   411.,   411.,   489.,   421.,   411.,   411.,   418.,
         411.,   411.,   411.,   411.,   408.,   408.,   408.,   411.,
         411.,   408.,   418.,   408.,   408.,   411.,   411.,   426.,
         408.,   411.,   547.,   411.,   677.,   411.,   701.,   411.,
         411.,   422.,   411.,   411.,   411.,   408.,   422.,   411.,
         411.,   411.,   411.,   411.,   411.,   411.,   453.,   411.,
         411.,   534.,   547.,   526.,   411.,   547.,   678.,   718.,
         419.,   419.,   521.,   408., 16082.,   772.,   499.,   423.,
         419.,   411.,   408.,   772.,   547.,   419.,   419.,   418.,
         521.,   411.,   474.,   419.,   470.,   419.,   499.,   411.,
         411.,   434.,   408.,   411.,   411.,   892.,   411., 16082.,
         411.,   411.,   411.,   711.,   411.,   408.,   435.,   436.,
         417.,   411.,   411.,   423.,   411.,   470.,   411.,   446.,
         411.,   411.,   411.,   958.,   836.,   411.,   892.,   673.,
      

In [239]:
"""
Factorization models for implicit feedback problems.
"""

import numpy as np

import torch

import torch.optim as optim

from torch.autograd import Variable

from helpers import _repr_model
from factorization._components import (_predict_process_features,
                                                 _predict_process_ids)
from losses import (adaptive_hinge_loss,
                              bpr_loss,
                              hinge_loss,
                              pointwise_loss)

from factorization.representations import (BilinearNet,
                                                     FeatureNet,
                                                     HybridContainer)

from sampling import sample_items
from torch_utils import cpu, gpu, set_seed


class ImplicitFactorizationModel(object):

    def __init__(self,
                 loss='pointwise',
                 embedding_dim=32,
                 n_iter=10,
                 batch_size=256,
                 l2=0.0,
                 learning_rate=1e-2,
                 optimizer_func=None,
                 use_cuda=False,
                 sparse=False,
                 random_state=None,
                 representation=None,
                 n_tastes=):

        assert loss in ('pointwise',
                        'bpr',
                        'hinge',
                        'adaptive_hinge')

        self._loss = loss
        self._embedding_dim = embedding_dim
        self._n_iter = n_iter
        self._learning_rate = learning_rate
        self._batch_size = batch_size
        self._l2 = l2
        self._use_cuda = use_cuda
        self._sparse = sparse
        self._optimizer_func = optimizer_func
        self._random_state = random_state or np.random.RandomState()

        self._num_users = None
        self._num_items = None
        self._net = None
        self._optimizer = None
        self._loss_func = None
        self._representation=representation

        set_seed(self._random_state.randint(-10**8, 10**8),
                 cuda=self._use_cuda)

    def __repr__(self):

        return _repr_model(self)

    @property
    def _initialized(self):
        return self._net is not None

    def _initialize(self, interactions):

        (self._num_users,
         self._num_items) = (interactions.num_users,
                             interactions.num_items)
        
        if self._representation=='mixture':
            latent_net = MixtureNet(self._num_users,
                                   self._num_items,
                                    self._embedding_dim)
        
        elif self._representation=='nonlinear_mixture':
            latent_net = NonlinearMixtureNet(self._num_users,
                                             self._num_items,
                                             self._embedding_dim)
        elif self._representation=='embedding_mixture':
            latent_net = EmbeddingMixtureNet(self._num_users,
                                            self._num_items,
                                            self._embedding_dim)

        else:
            latent_net = BilinearNet(self._num_users,
                                     self._num_items,
                                     self._embedding_dim,
                                     sparse=self._sparse)
            
            

        if interactions.num_user_features():
            user_net = FeatureNet(interactions.num_user_features(),
                                  self._embedding_dim)
        else:
            user_net = None

        if interactions.num_context_features():
            context_net = FeatureNet(interactions.num_context_features(),
                                     self._embedding_dim)
        else:
            context_net = None

        if interactions.num_item_features():
            item_net = FeatureNet(interactions.num_item_features(),
                                  self._embedding_dim)
        else:
            item_net = None

        self._net = gpu(HybridContainer(latent_net,
                                        user_net,
                                        context_net,
                                        item_net),
                        self._use_cuda)

        if self._optimizer_func is None:
            self._optimizer = optim.Adam(
                self._net.parameters(),
                weight_decay=self._l2,
                lr=self._learning_rate
            )
        else:
            self._optimizer = self._optimizer_func(self._net.parameters())

        if self._loss == 'pointwise':
            self._loss_func = pointwise_loss
        elif self._loss == 'bpr':
            self._loss_func = bpr_loss
        elif self._loss == 'hinge':
            self._loss_func = hinge_loss
        else:
            self._loss_func = adaptive_hinge_loss

    def _check_input(self, user_ids, item_ids, allow_items_none=False):

        if isinstance(user_ids, int):
            user_id_max = user_ids
        else:
            user_id_max = user_ids.max()

        if user_id_max >= self._num_users:
            raise ValueError('Maximum user id greater '
                             'than number of users in model.')

        if allow_items_none and item_ids is None:
            return

        if isinstance(item_ids, int):
            item_id_max = item_ids
        else:
            item_id_max = item_ids.max()

        if item_id_max >= self._num_items:
            raise ValueError('Maximum item id greater '
                             'than number of items in model.')

    def fit(self, interactions, verbose=False):

        user_ids = interactions.user_ids.astype(np.int64)
        item_ids = interactions.item_ids.astype(np.int64)

        if not self._initialized:
            self._initialize(interactions)

        self._check_input(user_ids, item_ids)

        for epoch_num in range(self._n_iter):

            interactions.shuffle(random_state=self._random_state)

            epoch_loss = 0.0

            for (minibatch_num,
                 minibatch) in enumerate(interactions
                                         .minibatches(use_cuda=self._use_cuda,
                                                      batch_size=self._batch_size)):

                minibatch = minibatch.torch(self._use_cuda).variable()

                positive_prediction = self._net(minibatch.user_ids,
                                                minibatch.item_ids,
                                                minibatch.user_features,
                                                minibatch.context_features,
                                                minibatch.item_features)
                
                if self._loss == 'adaptive_hinge':
                    negative_prediction = [self._get_negative_prediction(minibatch)
                                           for _ in range(5)]
                else:
                    negative_prediction = self._get_negative_prediction(minibatch)

                self._optimizer.zero_grad()

                loss = self._loss_func(
                    positive_prediction,
                    negative_prediction,
                    weights=minibatch.weights
                )

                epoch_loss += loss.data[0]

                loss.backward()
                self._optimizer.step()

            epoch_loss /= minibatch_num + 1

            if verbose:
                print('Epoch {}: loss {}'.format(epoch_num, epoch_loss))

    def _get_negative_prediction(self, minibatch):

        negative_items = sample_items(
            self._num_items,
            len(minibatch),
            random_state=self._random_state)
        
        negative_var = gpu(torch.from_numpy(negative_items), self._use_cuda)

        negative_prediction = self._net(minibatch.user_ids,
                                        negative_var,
                                        minibatch.user_features,
                                        minibatch.context_features,
                                        minibatch.item_features)

        return negative_prediction
    
    def predict(self, user_ids, item_ids=None,
            user_features=None,
            context_features=None,
            item_features=None):


        self._check_input(user_ids, item_ids, allow_items_none=True)
        self._net.train(False)

        user_ids, item_ids = _predict_process_ids(user_ids, item_ids,
                                                  self._num_items,
                                                  self._use_cuda)

        (user_features,
         context_features,
         item_features) = _predict_process_features(user_features,
                                                    context_features,
                                                    item_features,
                                                    len(item_ids),
                                                    self._use_cuda)

        out = self._net(user_ids,
                        item_ids,
                        user_features,
                        context_features,
                        item_features)

        return cpu(out.data).numpy().flatten()

### Hybrid containers, feature nets, and mixture representations

In [304]:
import random

import torch

import torch.nn as nn
import torch.nn.init
import torch.nn.functional as F

from torch.autograd import Variable

from spotlight.layers import ScaledEmbedding, ZeroEmbedding

"""
Classes defining user and item latent representations in
factorization models.
"""

import torch.nn as nn
import torch.nn.functional as F

from spotlight.layers import ScaledEmbedding, ZeroEmbedding


class HybridContainer(nn.Module):

    def __init__(self,
                 latent_module,
                 user_module=None,
                 context_module=None,
                 item_module=None):

        super(HybridContainer, self).__init__()

        self.latent = latent_module
        self.user = user_module
        self.context = context_module
        self.item = item_module

    def forward(self, user_ids,
                item_ids,
                user_features=None,
                context_features=None,
                item_features=None):

        user_representation, user_bias = self.latent.user_representation(user_ids)
        item_representation, item_bias = self.latent.item_representation(item_ids)

        if self.user is not None:
            user_representation += self.user(user_features)
        if self.context is not None:
            user_representation += self.context(context_features)
        if self.item is not None:
            item_representation += self.item(item_features)

        dot = (user_representation * item_representation).sum(1)

        return dot + user_bias + item_bias


class FeatureNet(nn.Module):

    def __init__(self, input_dim, output_dim, bias=False, nonlinearity='tanh'):

        super(FeatureNet, self).__init__()

        if nonlinearity == 'tanh':
            self.nonlinearity = F.tanh
        elif nonlinearity == 'relu':
            self.nonlinearity = F.relu
        elif nonlinearity == 'sigmoid':
            self.nonlinearity = F.sigmoid
        elif nonlinearity == 'linear':
            self.nonlinearity = lambda x: x
        else:
            raise ValueError('Nonlineariy must be one of '
                             '(tanh, relu, sigmoid, linear)')

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.fc_1 = nn.Linear(self.input_dim,
                              self.output_dim,
                              bias=bias)

    def forward(self, features):

        return self.nonlinearity(self.fc_1(features))


class BilinearNet(nn.Module):
    """
    Bilinear factorization representation.
    Encodes both users and items as an embedding layer; the score
    for a user-item pair is given by the dot product of the item
    and user latent vectors.
    Parameters
    ----------
    num_users: int
        Number of users in the model.
    num_items: int
        Number of items in the model.
    embedding_dim: int, optional
        Dimensionality of the latent representations.
    sparse: boolean, optional
        Use sparse gradients.
    """

    def __init__(self, num_users, num_items, embedding_dim=32, sparse=False):

        super(BilinearNet, self).__init__()

        self.embedding_dim = embedding_dim

        self.user_embeddings = ScaledEmbedding(num_users, embedding_dim,
                                               sparse=sparse)
        self.item_embeddings = ScaledEmbedding(num_items, embedding_dim,
                                               sparse=sparse)
        self.user_biases = ZeroEmbedding(num_users, 1, sparse=sparse)
        self.item_biases = ZeroEmbedding(num_items, 1, sparse=sparse)

    def user_representation(self, user_ids):

        user_embedding = self.user_embeddings(user_ids)
        user_embedding = user_embedding.view(-1, self.embedding_dim)

        user_bias = self.user_biases(user_ids).view(-1, 1)

        return user_embedding, user_bias

    def item_representation(self, item_ids):

        item_embedding = self.item_embeddings(item_ids)
        item_embedding = item_embedding.view(-1, self.embedding_dim)

        item_bias = self.item_biases(item_ids).view(-1, 1)

        return item_embedding, item_bias

    def forward(self, user_representation, user_bias, item_representation, item_bias):
        """
        Compute the forward pass of the representation.
        Parameters
        ----------
        user_ids: tensor
            Tensor of user indices.
        item_ids: tensor
            Tensor of item indices.
        Returns
        -------
        predictions: tensor
            Tensor of predictions.
        """

        dot = (user_representation * item_representation).sum(1)

        return dot + user_bias + item_bias

class MixtureNet(nn.Module):
    """
    Bilinear factorization representation.
    Encodes both users and items as an embedding layer; the score
    for a user-item pair is given by the dot product of the item
    and user latent vectors.
    Parameters
    ----------
    num_users: int
        Number of users in the model.
    num_items: int
        Number of items in the model.
    embedding_dim: int, optional
        Dimensionality of the latent representations.
    user_embedding_layer: an embedding layer, optional
        If supplied, will be used as the user embedding layer
        of the network.
    item_embedding_layer: an embedding layer, optional
        If supplied, will be used as the item embedding layer
        of the network.
    sparse: boolean, optional
        Use sparse gradients.
    """

    def __init__(self, num_users, num_items, embedding_dim=32,
                 projection_scale=1.0,
                 num_components=4):

        super(MixtureNet, self).__init__()

        self.embedding_dim = embedding_dim
        self.num_components = num_components
        self.projection_scale = projection_scale

        self.user_embeddings = ScaledEmbedding(num_users, embedding_dim)
        self.item_embeddings = ScaledEmbedding(num_items, embedding_dim)

        self.user_biases = ZeroEmbedding(num_users, 1)
        self.item_biases = ZeroEmbedding(num_items, 1)

        self.taste_projection = nn.Linear(embedding_dim,
                                          embedding_dim * self.num_components, bias=False)
        self.attention_projection = nn.Linear(embedding_dim,
                                              embedding_dim * self.num_components, bias=False)

        for layer in (self.taste_projection, self.attention_projection):
            torch.nn.init.xavier_normal(layer.weight, self.projection_scale)

    def user_representation(self, user_ids):

        user_embedding = self.user_embeddings(user_ids).squeeze()
        user_bias = self.user_biases(user_ids).squeeze()

        return user_embedding, user_bias

    def item_representation(self, item_ids):

        item_embedding = self.item_embeddings(item_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        return item_embedding, item_bias

    def forward(self, user_ids, item_ids):

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        batch_size, embedding_size = item_embedding.size()

        user_tastes = (self.taste_projection(user_embedding)
                       .resize(batch_size,
                               self.num_components,
                               embedding_size))
        user_attention = (self.attention_projection(user_embedding)
                          .resize(batch_size,
                                  self.num_components,
                                  embedding_size))
        user_attention = user_attention #  * user_embedding.unsqueeze(1).expand_as(user_tastes)

        attention = (F.softmax((user_attention *
                                item_embedding.unsqueeze(1).expand_as(user_attention))
                               .sum(2)).unsqueeze(2).expand_as(user_attention))
        weighted_preference = (user_tastes * attention).sum(1)

        dot = (weighted_preference * item_embedding).sum(1)

        user_bias = self.user_biases(user_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        if random.random() < 0.5:
            print('User tastes', user_tastes[0][0].abs().mean().data[0])
            print('Tastes', weighted_preference.abs().mean().data[0])
        #     print('Attention', (user_attention * item_embedding).sum(2).max().data[0])
        #     print('Softmax', attention.max(1)[0].mean().data[0])
            print('Preference', preference.max(1)[0].mean().data[0])
        #     print('Prediction', weighted_preference.mean().data[0])
        #     print('Biases', user_bias.max().data[0], item_bias.max().data[0])

        return dot + user_bias + item_bias


class MixtureComponent(nn.Module):

    def __init__(self, embedding_dim, num_components):

        super(MixtureComponent, self).__init__()

        self.embedding_dim = embedding_dim
        self.num_components = num_components

        self.fc_1 = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.fc_2 = nn.Linear(embedding_dim, embedding_dim, bias=False)

        self.taste_projection = nn.Linear(embedding_dim,
                                          embedding_dim * num_components,
                                          bias=False)
        self.attention_projection = nn.Linear(embedding_dim,
                                              embedding_dim * num_components,
                                              bias=False)

    def forward(self, x):

        batch_size, embedding_size = x.size()

        x = F.relu(self.fc_1(x))
        x = F.relu(self.fc_2(x))

        user_tastes = (self.taste_projection(x)
                       .resize(batch_size,
                               self.num_components,
                               embedding_size))
        user_attention = (self.attention_projection(x)
                          .resize(batch_size,
                                  self.num_components,
                                  embedding_size))

        return user_tastes, user_attention


class NonlinearMixtureNet(nn.Module):
    """
    Bilinear factorization representation.
    Encodes both users and items as an embedding layer; the score
    for a user-item pair is given by the dot product of the item
    and user latent vectors.
    Parameters
    ----------
    num_users: int
        Number of users in the model.
    num_items: int
        Number of items in the model.
    embedding_dim: int, optional
        Dimensionality of the latent representations.
    user_embedding_layer: an embedding layer, optional
        If supplied, will be used as the user embedding layer
        of the network.
    item_embedding_layer: an embedding layer, optional
        If supplied, will be used as the item embedding layer
        of the network.
    sparse: boolean, optional
        Use sparse gradients.
    """

    def __init__(self, num_users, num_items, embedding_dim=32,
                 num_components=4):

        super(NonlinearMixtureNet, self).__init__()

        self.embedding_dim = embedding_dim
        self.num_components = num_components

        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)

        self.user_biases = ZeroEmbedding(num_users, 1)
        self.item_biases = ZeroEmbedding(num_items, 1)

        self.mixture = MixtureComponent(embedding_dim, num_components)
        
    def user_representation(self, user_ids):

        user_embedding = self.user_embeddings(user_ids).squeeze()
        user_bias = self.user_biases(user_ids).squeeze()

        return user_embedding, user_bias

    def item_representation(self, item_ids):

        item_embedding = self.item_embeddings(item_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        return item_embedding, item_bias

    def forward(self, user_ids, item_ids):
        """
        Compute the forward pass of the representation.
        Parameters
        ----------
        user_ids: tensor
            Tensor of user indices.
        item_ids: tensor
            Tensor of item indices.
        Returns
        -------
        predictions: tensor
            Tensor of predictions.
        """

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        batch_size, embedding_size = item_embedding.size()

        user_tastes, user_attention = self.mixture(user_embedding)
        item_embedding = item_embedding.unsqueeze(1).expand_as(user_attention)

        attention = F.softmax((user_attention * item_embedding).sum(2))

        preference = ((user_tastes * item_embedding)
                      .sum(2))
        weighted_preference = (attention * preference).sum(1).squeeze()

        user_bias = self.user_biases(user_ids).squeeze()
        item_bias = self.item_biases(item_ids).squeeze()

        return weighted_preference + user_bias + item_bias

In [305]:
train,test = random_train_test_split(feature_interactions)

model = ImplicitFactorizationModel(n_iter=25,
                                   loss='adaptive_hinge',
                                   l2=-1.252542352e-10,
                                   representation='mixture')
model.fit(train,verbose=True)


/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:204: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:214: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0: loss 0.8663638234138489
Epoch 1: loss 0.6864293217658997
Epoch 2: loss 0.4440964162349701
Epoch 3: loss 0.3187476694583893
Epoch 4: loss 0.2590058147907257
Epoch 5: loss 0.2247215211391449
Epoch 6: loss 0.21046796441078186
Epoch 7: loss 0.2007794827222824
Epoch 8: loss 0.18609416484832764
Epoch 9: loss 0.1828065812587738
Epoch 10: loss 0.17679086327552795
Epoch 11: loss 0.1738606095314026
Epoch 12: loss 0.17146216332912445
Epoch 13: loss 0.1661866158246994
Epoch 14: loss 0.16833245754241943
Epoch 15: loss 0.16007539629936218
Epoch 16: loss 0.16719187796115875
Epoch 17: loss 0.15949228405952454
Epoch 18: loss 0.16238249838352203
Epoch 19: loss 0.1591346710920334
Epoch 20: loss 0.15744638442993164
Epoch 21: loss 0.15526330471038818
Epoch 22: loss 0.15154650807380676
Epoch 23: loss 0.15564003586769104


KeyboardInterrupt: 

In [303]:
from evaluation import mrr_score
mrr = mrr_score(model, test)
print('mean mrr',mrr.mean())

mean mrr 0.09999340549235591


In [292]:
model

<ImplicitFactorizationModel: HybridContainer(
  (latent): MixtureNet(
    (user_embeddings): ScaledEmbedding(13508, 32)
    (item_embeddings): ScaledEmbedding(314, 32)
    (user_biases): ZeroEmbedding(13508, 1)
    (item_biases): ZeroEmbedding(314, 1)
    (taste_projection): Linear(in_features=32, out_features=128, bias=False)
    (attention_projection): Linear(in_features=32, out_features=128, bias=False)
  )
  (item): FeatureNet(
    (fc_1): Linear(in_features=314, out_features=32, bias=False)
  )
)>

In [293]:
for i in model._net.parameters():
    print(i)

Parameter containing:
tensor([[-5.2386e-01,  1.3071e-01,  1.5399e-01,  ..., -3.5471e-01,
          1.0408e-01,  2.4395e-01],
        [ 6.9572e-01, -1.9793e-01,  7.5319e-01,  ..., -7.4743e-01,
         -8.7165e-02, -2.4112e-01],
        [-9.8150e-02, -5.7035e-01,  5.8710e-01,  ..., -1.7389e-01,
         -3.4538e-01, -1.3546e-01],
        ...,
        [-1.4044e-01, -2.8648e-01,  5.2763e-01,  ...,  3.5332e-01,
         -6.9260e-01,  1.8774e-01],
        [-5.9483e-02, -1.5412e-02,  8.6411e-02,  ...,  8.8151e-02,
          5.6804e-03, -4.6847e-02],
        [-5.2888e-02, -2.1481e-01,  9.7891e-02,  ...,  2.4684e-01,
         -7.2911e-01, -8.4581e-01]])
Parameter containing:
tensor([[ 0.2138, -0.3615, -0.0054,  ...,  0.3856,  0.1256,  0.5986],
        [ 0.0830, -0.1182, -0.6525,  ...,  0.3090,  0.0470,  0.0448],
        [ 1.2524, -0.9382,  0.9913,  ..., -0.1498,  0.2701, -0.3964],
        ...,
        [ 0.6772,  0.5899, -0.2344,  ..., -0.2411,  0.0709, -0.4926],
        [-0.2198,  0.5926, -0.8

In [294]:
learned = list(model._net.parameters())

user_embeddings = learned[0]
item_embeddings = learned[1]

user_embeddings = list(zip(user_embeddings,range(0,feature_interactions.num_users)))
item_embeddings = list(zip(item_embeddings,range(0,feature_interactions.num_items)))


In [295]:
external_userIds[13495]

14613

In [296]:
user_dict = {}

for emb,u in user_embeddings:
    try:
        user_dict[external_userIds[u]] = emb
    except:
        print(u)

user_dict

{365916: tensor([-0.5239,  0.1307,  0.1540, -0.1801,  0.2830,  0.5017, -0.1257,
          0.1109, -0.0558, -0.2662, -0.6579, -0.4219, -0.1363, -0.2677,
         -0.1844,  0.3359,  0.3970, -0.2466, -0.6956, -0.3563,  0.3662,
         -0.4512,  0.0463,  0.1006,  0.1692, -0.3704,  0.4684, -0.0817,
         -0.0011, -0.3547,  0.1041,  0.2439]),
 365914: tensor([ 0.6957, -0.1979,  0.7532,  0.9608,  0.3157, -0.3246,  0.8590,
         -0.2922, -0.6546, -0.5066,  0.1169, -0.0028,  0.6806, -0.0405,
         -0.6999, -0.1834,  0.1372,  0.0545, -0.3122, -0.2276, -0.3038,
          1.0059,  0.1069,  0.0302, -0.3475,  0.8909, -0.1528, -0.2188,
         -0.3641, -0.7474, -0.0872, -0.2411]),
 365427: tensor([-0.0981, -0.5703,  0.5871, -0.5539, -0.8382, -0.3817,  0.4946,
         -0.3668, -1.1195,  0.1275,  0.5885, -1.0615,  0.5117,  0.0698,
         -0.7361,  0.2956, -0.0940, -0.2627, -0.1002,  0.5616,  0.0801,
          0.1644,  0.1868, -0.2259,  0.0803,  0.7102,  0.1743,  0.6110,
          1.1460, 

In [297]:
item_dict = {}

for emb,i in item_embeddings:
    try:
        item_dict[external_itemIds[i]] = emb
    except:
        print(u)

item_dict

{1458: tensor([ 0.2138, -0.3615, -0.0054, -0.6235,  0.8590, -0.0496, -0.1164,
         -0.8648,  0.8847,  0.5591, -0.9906, -0.5988, -0.4529, -0.6874,
         -1.0035,  0.1663,  0.2101, -0.6847, -0.3887, -0.1075,  0.8879,
         -0.5108,  0.3991, -0.1783, -0.5928, -0.9381,  0.3967,  0.2612,
          0.2555,  0.3856,  0.1256,  0.5986]),
 1247: tensor([ 0.0830, -0.1182, -0.6525, -0.4600, -0.3404, -0.0449, -0.4220,
          0.0550, -0.1938,  0.4163, -0.4333, -0.4977,  0.0868, -0.3862,
         -0.3119,  0.4540,  0.6036,  0.4827, -0.2745, -0.7329,  0.5281,
         -0.1064, -0.8343,  0.5941, -0.3133,  0.1888,  0.3467,  0.3951,
          0.6902,  0.3090,  0.0470,  0.0448]),
 1575: tensor([ 1.2524, -0.9382,  0.9913,  0.6612,  0.4702,  0.1862,  0.9952,
         -0.3785, -0.1620, -0.2301,  0.6684, -0.2857,  1.0069,  0.8692,
         -0.4496, -0.3127,  0.2215,  0.1971, -0.8351, -0.8481, -0.0118,
          0.9118, -0.1394,  0.5556, -0.2095,  0.3344, -0.0378,  0.0873,
          0.0500, -0.149

In [298]:
fnc = lambda x:x[0]

fnc(item_embeddings)

(tensor([ 0.2138, -0.3615, -0.0054, -0.6235,  0.8590, -0.0496, -0.1164,
         -0.8648,  0.8847,  0.5591, -0.9906, -0.5988, -0.4529, -0.6874,
         -1.0035,  0.1663,  0.2101, -0.6847, -0.3887, -0.1075,  0.8879,
         -0.5108,  0.3991, -0.1783, -0.5928, -0.9381,  0.3967,  0.2612,
          0.2555,  0.3856,  0.1256,  0.5986]), 0)

In [323]:
userId=79593

user = list(zip(range(feature_interactions.num_items),[user_dict[userId].dot(emb) for emb in learned[1]]))

In [324]:
preds = sorted(user,key=lambda x:x[1],reverse=True)

In [325]:
city_data[city_data['userId']==userId]

,userId,email,merchant,city,branchId,suburb
213521,79593,tranmplinh@gmail.com,Mad Mex,Sydney,1043,411.0
213522,79593,tranmplinh@gmail.com,Mad Mex,Sydney,1045,411.0
213523,79593,tranmplinh@gmail.com,iDarts Zen,Sydney,1463,411.0
213524,79593,tranmplinh@gmail.com,Messina,Sydney,1114,421.0
213525,79593,tranmplinh@gmail.com,Din Tai Fung,Sydney,1534,411.0
213526,79593,tranmplinh@gmail.com,Sushi Rap,Sydney,1457,411.0
213527,79593,tranmplinh@gmail.com,Oh! Matcha,Sydney,1458,411.0
213528,79593,tranmplinh@gmail.com,Gin Im Thai,Sydney,661,422.0
213529,79593,tranmplinh@gmail.com,Banh and Butter,Sydney,1261,411.0
213530,79593,tranmplinh@gmail.com,Messina,Sydney,1119,499.0


In [326]:
preds = sorted(user,key=lambda x:x[1],reverse=True)

for i in range(20):
    branchId = external_itemIds[preds[i][0]]
    x = city_data[city_data['branchId']==branchId]
    merchant = x.merchant.unique()
    suburb = x.suburb.unique()
    print(preds[i][1],' ------ ',merchant[0],': ',str(suburb[0]))
    

tensor(3.1047)  ------  Fratelli Fresh :  411.0
tensor(2.8827)  ------  Mad Mex :  411.0
tensor(2.6585)  ------  Cafe Miiso :  490.0
tensor(2.5669)  ------  Pasta Pantry :  462.0
tensor(2.5239)  ------  The Sandwich Shop :  408.0
tensor(2.4376)  ------  South Dowling Sandwiches :  422.0
tensor(2.3977)  ------  Fratelli Fresh :  411.0
tensor(2.1190)  ------  Sushi Rap :  411.0
tensor(2.1163)  ------  Coco Cubano :  673.0
tensor(2.0703)  ------  The Choc Pot :  411.0
tensor(2.0533)  ------  iDarts Zen :  411.0
tensor(1.8700)  ------  Ichi-ban Boshi :  411.0
tensor(1.8560)  ------  Piccolo Me :  411.0
tensor(1.8256)  ------  Salt Meats Cheese :  411.0
tensor(1.8191)  ------  Zushi :  422.0
tensor(1.7095)  ------  Roll'd :  411.0
tensor(1.6573)  ------  Edition Coffee Roasters :  408.0
tensor(1.6343)  ------  The Sandwich Shop :  423.0
tensor(1.5967)  ------  Coco Cubano :  419.0
tensor(1.5734)  ------  Sandoitchi :  422.0


In [327]:
for i in learned[1]:
    print(i.shape)

torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32

In [328]:
taste_projection = nn.Linear(32,32 * 2, bias=False)

In [331]:
tastes = taste_projection(user_dict[userId]).resize(4,32)

In [332]:
tastes

tensor([[ 0.2232,  0.0434,  0.1107, -0.1028, -0.1870,  0.0368,  0.1808,
         -0.0303, -0.0364,  0.1985, -0.0323, -0.1388, -0.0401, -0.0204,
         -0.1173,  0.0327,  0.0521,  0.3920, -0.1612, -0.2003, -0.1498,
          0.0199, -0.0394,  0.0879, -0.0095,  0.0221, -0.1435, -0.0598,
          0.3552, -0.2786, -0.0430, -0.0279],
        [ 0.0368, -0.1037, -0.1588, -0.0226, -0.1469, -0.2331, -0.0920,
          0.0502, -0.0088,  0.0690,  0.3283, -0.2063,  0.1889,  0.1864,
         -0.2723, -0.1098,  0.0459,  0.1155, -0.2275, -0.0445,  0.1792,
          0.0957, -0.0318, -0.2210,  0.0292,  0.2989,  0.2405,  0.1759,
          0.3067, -0.1619, -0.1393, -0.1894],
        [ 0.2152, -0.1092,  0.0941, -0.0033, -0.1164,  0.2285,  0.3593,
         -0.2936,  0.0860, -0.2040, -0.0752,  0.0628,  0.0017, -0.2387,
          0.0742, -0.0698,  0.2363, -0.1536,  0.5754, -0.0866, -0.2760,
         -0.0563, -0.2214, -0.0251, -0.0304,  0.1963, -0.0764,  0.1695,
         -0.1339,  0.0543,  0.0320, -0.1636]

In [333]:
preds = []
for taste in tastes:
    preds+=[list(zip(range(feature_interactions.num_items),[taste.dot(emb) for emb in learned[1]]))]
    
preds

[[(0, tensor(-0.1868)),
  (1, tensor(0.7062)),
  (2, tensor(0.8894)),
  (3, tensor(1.00000e-02 *
          -7.7101)),
  (4, tensor(0.7082)),
  (5, tensor(0.5149)),
  (6, tensor(0.7489)),
  (7, tensor(0.1357)),
  (8, tensor(1.00000e-02 *
          -2.0798)),
  (9, tensor(-0.3900)),
  (10, tensor(1.00000e-02 *
          5.5985)),
  (11, tensor(0.1218)),
  (12, tensor(0.1321)),
  (13, tensor(0.5388)),
  (14, tensor(1.00000e-02 *
          -6.5306)),
  (15, tensor(0.8336)),
  (16, tensor(-0.4373)),
  (17, tensor(1.00000e-02 *
          -9.6863)),
  (18, tensor(1.00000e-02 *
          7.3314)),
  (19, tensor(-0.1896)),
  (20, tensor(-0.3100)),
  (21, tensor(0.6042)),
  (22, tensor(-1.1452)),
  (23, tensor(-0.2595)),
  (24, tensor(-0.5955)),
  (25, tensor(0.3714)),
  (26, tensor(1.00000e-02 *
          -1.3704)),
  (27, tensor(0.3161)),
  (28, tensor(-0.2037)),
  (29, tensor(-0.1124)),
  (30, tensor(0.2448)),
  (31, tensor(-0.7630)),
  (32, tensor(1.00000e-02 *
          3.7244)),
  (33, ten

In [334]:
count = 0
for recs in preds:
    recs = sorted(recs,key=lambda x:x[1],reverse=True)
    count += 1
    print('Taste profile: ',count)
    for i in range(10):
        branchId = external_itemIds[recs[i][0]]
        x = city_data[city_data['branchId']==branchId]
        merchant = x.merchant.unique()
        suburb = x.suburb.unique()
        print(recs[i][1],' ------ ',merchant[0],': ',str(suburb[0]))
    print('\n')


Taste profile:  1
tensor(1.2532)  ------  Crafty Cuts :  446.0
tensor(1.1906)  ------  QQ Tea :  772.0
tensor(1.1606)  ------  The Char Rotisserie :  453.0
tensor(1.0956)  ------  Roll'd :  411.0
tensor(1.0915)  ------  Black Label :  411.0
tensor(1.0151)  ------  Pasta Pantry :  462.0
tensor(0.9895)  ------  Condor Japanese Restaurant :  411.0
tensor(0.9825)  ------  Poked :  411.0
tensor(0.9512)  ------  Two Sticks :  16082.0
tensor(0.9318)  ------  Pasta Pantry :  411.0


Taste profile:  2
tensor(1.8634)  ------  Tontaro Ramen :  411.0
tensor(1.6821)  ------  Poked :  411.0
tensor(1.5234)  ------  Enjoy Cafe by Enjoy Mie :  411.0
tensor(1.3268)  ------  Din Tai Fung :  421.0
tensor(1.3234)  ------  BL Burgers :  422.0
tensor(1.2995)  ------  Din Tai Fung :  958.0
tensor(1.2989)  ------  Bar Cupola :  411.0
tensor(1.2329)  ------  The Little Snail :  421.0
tensor(1.2179)  ------  Mr Crackles :  422.0
tensor(1.2102)  ------  Three Williams :  434.0


Taste profile:  3
tensor(1.7489)  